# Q-LoRA

### 한국어 QA 시스템 최적화

##### 0. 환경 설정

In [ ]:
!pip install peft datasets transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 28.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 44.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 186.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 173.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.2 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [ ]:
!pip install typing_extensions==4.7.1 --upgrade

  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.8.0.dev20250319+cu128 requires typing-extensions>=4.10.0, but you have typing-extensions 4.7.1 which is incompatible.


In [ ]:
from huggingface_hub import login

login(token="hf_xxx")

In [6]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

##### 1. 모델 로드

In [ ]:
model_name = 'NCSOFT/Llama-VARCO-8B-Instruct'

In [ ]:
# 4-bit 양자화된 모델 로드를 위한 설정
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                    
    bnb_4bit_quant_type='nf4',            
    bnb_4bit_use_double_quant=True,       
    bnb_4bit_compute_dtype=torch.bfloat16 
)

In [ ]:
# 토크나이저 및 모델 로드
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

##### 2. 데이터 준비

In [11]:
# 데이터셋 로드
from datasets import load_dataset

dataset = load_dataset('KorQuAD/squad_kor_v1')

README.md: 0.00B [00:00, ?B/s]

squad_kor_v1/train-00000-of-00001.parque(…):   0%|          | 0.00/11.6M [00:00<?, ?B/s]

squad_kor_v1/validation-00000-of-00001.p(…):   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [12]:
dataset['train'][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

In [ ]:
# 데이터 전처리 함수
def preprocess_data(examples):

    inputs = ["질문: " + q + "\n문맥: " + c for q, c in zip(examples["question"], examples["context"])]
    answer_texts = [a["text"][0] if len(a["text"]) > 0 else "" for a in examples["answers"]]

    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    labels = tokenizer(
        answer_texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )["input_ids"]

    max_length = model_inputs["input_ids"].shape[1]
    labels = labels[:, :max_length]

    labels[labels == tokenizer.pad_token_id] = -100
    
    model_inputs["labels"] = labels    
    
    return model_inputs

In [15]:
# 데이터셋 전처리 적용
train_dataset = dataset["train"].map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
val_dataset = dataset["validation"].map(preprocess_data, batched=True, remove_columns=dataset["validation"].column_names)

Map:   0%|          | 0/60407 [00:00<?, ? examples/s]

Map:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [16]:
print(tokenizer.decode(train_dataset[0]["input_ids"], skip_special_tokens=True))

질문: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
문맥: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.
답변:


##### 3. 모델 학습 준비

In [ ]:
# LoRA 설정
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    target_modules=['q_proj', 'v_proj'],
    task_type='CAUSAL_LM'
)

In [19]:
# LoRA 적용
from peft import get_peft_model

model = get_peft_model(base_model, lora_config)

model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


In [20]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./q_lora_korqa',
    save_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    # num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
    report_to='none'
)

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_631/3977714251.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


##### 4. 모델 학습 (Fine-tuning)

In [ ]:
# 시간 오래 걸림
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 0}.


Step,Training Loss
100,5.817500
200,5.204500
300,5.202400
400,5.158700
500,5.077900
600,5.117800
700,5.092600
800,5.108500
900,5.119700
1000,5.118200


TrainOutput(global_step=5664, training_loss=5.036699553667488, metrics={'train_runtime': 20306.7912, 'train_samples_per_second': 8.924, 'train_steps_per_second': 0.279, 'total_flos': 4.17997065816952e+18, 'train_loss': 5.036699553667488, 'epoch': 3.0})

##### 5. 학습된 모델 활용

(1) 모델 추론

In [ ]:
# 설정 저장
from transformers import AutoConfig

trained_model_path = "./q_lora_korqa/checkpoint-xxxx"
trained_model_path = "./q_lora_korqa/checkpoint-5664"

config = AutoConfig.from_pretrained(model_name)
config.save_pretrained(trained_model_path)

In [26]:
# Base model + 학습 결과
from peft import PeftModel
from transformers import pipeline

base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
model = PeftModel.from_pretrained(base_model, trained_model_path)

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [27]:
question = "바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?"
context = "1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. "

input_text = f"질문: {question}\n문맥: {context}"

output = qa_pipeline(input_text, max_new_tokens=50, temperature=0.7, top_p=0.8)

print(output)

[{'generated_text': '질문: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?\n문맥: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 장르의명리르제명이르리로르 지,르르르2기리르,에12드기기일르 일의로,로우제일르르루루1르레프일'}]


(2) HuggingFace Hub

In [28]:
# LoRA 어댑터 or 병합 모델 저장
model.save_pretrained("./q_lora_korqa")
tokenizer.save_pretrained("./q_lora_korqa")

# HuggingFace 업로드
model.push_to_hub("squirreln/q_lora_korqa")
tokenizer.push_to_hub("squirreln/q_lora_korqa")

No files have been modified since last commit. Skipping to prevent empty commit.


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/squirreln/q_lora_korqa/commit/2a519f9a79e4b8470a16b9f846648e7860568afa', commit_message='Upload tokenizer', commit_description='', oid='2a519f9a79e4b8470a16b9f846648e7860568afa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/squirreln/q_lora_korqa', endpoint='https://huggingface.co', repo_type='model', repo_id='squirreln/q_lora_korqa'), pr_revision=None, pr_num=None)

In [ ]:
# HuggingFace에서 로드해서 사용 (Base Model, Adapter 각각 load)
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 베이스 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    "NCSOFT/Llama-VARCO-8B-Instruct",  # 원래 쓰던 base model
    torch_dtype="auto",
    device_map="auto"
)

# LoRA 어댑터 로드
model = PeftModel.from_pretrained(
    base_model,
    "squirreln/q_lora_korqa_"  # HuggingFace repo id
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("squirreln/q_lora_korqa_")

# 예시
inputs = tokenizer("바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/348 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?)))))))))))))))))) ))))      ))         의  의  일일  
